# Was sind die größten und komplexesten Methoden?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [1]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [2]:
import pandas as pd

query ="MATCH (t:Type)-[:HAS_SOURCE]->(f), (t)-[:DECLARES]->(m:Method) RETURN DISTINCT t.fqn as type, m.signature as signature,  m.effectiveLineCount as loc, m.cyclomaticComplexity as complexity"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [3]:
df.head()

,complexity,loc,signature,type
0,1.0,5.0,void correctCallbacksCalled(),org.junit.rules.TestWatcherTest$Callbacks
1,1.0,1.0,void <init>(),org.junit.rules.TestWatcherTest$Callbacks
2,1.0,21.0,void testWatcherDoesNotModifyResult(),org.junit.rules.TestWatcherTest$Callbacks
3,1.0,1.0,org.junit.rules.TestRule access$0(),org.junit.rules.TestWatcherTest$Callbacks
4,1.0,17.0,java.lang.Object[][] parameters(),org.junit.rules.TestWatcherTest$Callbacks


Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Anzahl der Quelltextzeilen (loc) mit der Methode ```sort_values``` sortiert und die ersten 10 Zeilen der Spalten ```loc```, ```signature``` und ```type``` im Dataframe ```loc_df``` abgebildet (https://www.data-science-architect.de/selektieren-von-daten-in-dataframes/).

In [4]:
# Sortiere das Dataframe nach der Anzahl der Quelltextzeilen (loc).
df = df.sort_values('loc',ascending=False)
# Extrahiere die Spalten loc, signature und type und speichere sie im Dataframe loc_df.
loc_df = df[['loc','signature','type']][0:10]
loc_df.head(10)

,loc,signature,type
1121,60.0,java.lang.Object[][] testsWithEventMatcher(),org.junit.rules.ErrorCollectorTest
1123,55.0,java.util.Collection testsWithEventMatcher(),org.junit.rules.ExpectedExceptionTest
937,35.0,"void arrayEquals(java.lang.String,java.lang.Ob...",org.junit.internal.ComparisonCriteria
3525,34.0,junit.framework.Test getTest(java.lang.String),junit.runner.BaseTestRunner
3477,29.0,java.lang.Object createTestUsingFieldInjection(),org.junit.runners.parameterized.BlockJUnit4Cla...
5731,28.0,void <clinit>(),org.junit.internal.runners.rules.RuleMemberVal...
3082,26.0,void setUp(),junit.tests.runner.StackFilterTest
3916,25.0,junit.framework.TestResult start(java.lang.Str...,junit.textui.TestRunner
3464,24.0,void validateFields(java.util.List),org.junit.runners.parameterized.BlockJUnit4Cla...
4604,24.0,void usesPassedInClassObject(),org.junit.tests.experimental.max.DescriptionTest


Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Komplexität (complexity) mit der Methode ```sort_values``` sortiert und die ersten 10 Zeilen der Spalten ```complexity```, ```signature``` und ```type``` im Dataframe ```complexity_df``` abgebildet (https://www.data-science-architect.de/selektieren-von-daten-in-dataframes/).

In [5]:
# Sortiere das Dataframe nach der zyklomatischen Komplexität (complexity).
df = df.sort_values('complexity',ascending=False)

# Extrahiere die Spalten complexity, signature und type und speichere sie im Dataframe complexity_df.
complexity_df = df[['complexity','signature','type']][0:10]
complexity_df.head(10)


,complexity,signature,type
937,18.0,"void arrayEquals(java.lang.String,java.lang.Ob...",org.junit.internal.ComparisonCriteria
5375,13.0,java.lang.String[] parseOptions(java.lang.Stri...,org.junit.runner.JUnitCommandLineParseResult
3916,13.0,junit.framework.TestResult start(java.lang.Str...,junit.textui.TestRunner
3517,12.0,void readPreferences(),junit.runner.BaseTestRunner
5939,12.0,void evaluate(),org.junit.internal.runners.statements.RunAfters
3162,12.0,"void execTest(java.lang.String,boolean)",junit.tests.runner.TextRunnerTest
3464,12.0,void validateFields(java.util.List),org.junit.runners.parameterized.BlockJUnit4Cla...
3529,11.0,java.lang.String processArguments(java.lang.St...,junit.runner.BaseTestRunner
5875,10.0,java.util.List getCauseStackTraceLines(java.la...,org.junit.internal.Throwables
5126,10.0,java.io.File newFolder(java.lang.String[]),org.junit.rules.TemporaryFolder


## 4. Visualisierung
Die größten und komplexesten Methoden werden jeweils mit einem horizontalen Balkendiagramm visualisiert.

In [6]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [7]:
# Erstelle Bar Chart für loc.
import pygal
bar_chart = pygal.HorizontalBar(show_legend=True, human_readable=True, fill=True, legend_at_bottom=True, legend_at_bottom_columns=2)
bar_chart.title = 'Größte Methoden'
for index, row in loc_df.iterrows():
    bar_chart.add(row['type'],[{"value": row['loc'], "label": row['signature']}])
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))

In [8]:
# Erstelle Bar Chart für complexity.
import pygal 
bar_chart = pygal.HorizontalBar(show_legend=True, human_readable=True, fill=True, legend_at_bottom=True, legend_at_bottom_columns=2)
bar_chart.title = 'Komplexeste Methoden'
for index, row in complexity_df.iterrows():
    bar_chart.add(row['type'],[{"value": row['complexity'], "label": row['signature']}])
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))